In [81]:
import pandas as pd

In [82]:
df = pd.read_csv('Merged_Data_202101_202303_v3.csv').drop(columns=['Unnamed: 0', 'feels_like'], axis = 1)

In [83]:
station = sorted(df['station_id'].unique())

In [84]:
# Kick the data after '2023-03-31 23:00:00'

df['Hour'] = pd.to_datetime(df['Hour'])
df = df[df['Hour'] <= '2023-03-31 23:00:00']

In [88]:
# Start and end
start_date = '2021-01-01 00:00:00'
end_date = '2023-03-31 23:00:00'

# Create a complete timestamp
date_range = pd.date_range(start=start_date, end=end_date, freq='h')

# Initial a new Dataframe
complete_data = pd.DataFrame()

for station_id in station:
    # Create a DataFrame for this station
    df_station = pd.DataFrame(date_range, columns=['Hour'])
    df_station['station_id'] = station_id
    
    # Assign 0 to 'Net_Flow', assume no flow initially
    df_station['Net_Flow'] = 0
    
    # Append the data for this station to the complete DataFrame
    complete_data = pd.concat([complete_data, df_station])

complete_data.reset_index(drop=True, inplace=True)

In [89]:
complete_data = pd.merge(complete_data, df[['Hour', 'station_id', 'Net_Flow']],
                          on=['Hour', 'station_id'], how='left', suffixes=('', '_orig'))

complete_data['Net_Flow'] = complete_data['Net_Flow_orig'].fillna(complete_data['Net_Flow'])
complete_data.drop(columns=['Net_Flow_orig'], inplace=True)

print(complete_data.head())

                 Hour  station_id  Net_Flow
0 2021-01-01 00:00:00           1       0.0
1 2021-01-01 01:00:00           1       0.0
2 2021-01-01 02:00:00           1       0.0
3 2021-01-01 03:00:00           1       0.0
4 2021-01-01 04:00:00           1       0.0


In [90]:
complete_data.to_csv('ar_data.csv', index = False)